# ORDER OF MODELING STEPS


This notebook follows the modeling framework outlined below.
1. Conceptualize Model (What do I want to model and why?)
2. Build model (figure out equations, write code)
3. Fit model to surrogate data (parameter and model recovery)

**CHECKPOINT: Only continue if the model and experiment can answer the question in theory, and if parameters and model are recoverable**

4. Fit model to participant data(1. parameter and model fit, 2. validate model)

**CHECKPOINT: only continue if the model can account for data.**
5. latent variable analysis and report results

Based on Prof. Musslick’s lecture slides in his class about cognitive modeling in the winter semester 2024.




## STEP 1) CONCEPTUALIZE MODEL — LOCK + CHECKLIST

### Locked intent and claim boundaries
- [x] Target claim: "Among the candidate models, under the pre-specified scoring + held-out evaluation + checks, model X best predicts choice+RT in this dataset."
- [x] Non-claim boundary: "X is the true brain model" is out of scope.
- [x] Scope is participant-wise (`P01`, `P02`, `P03`) with three candidate models.

### Locked hazard input interpretation
- [x] Hazard input is fixed to `subjective_h_snapshot`.
- [x] Constraint: treat hazard input as past-only / externally fixed during fit and evaluation.
- [x] Required caveat sentence remains mandatory in final reporting.


## STEP 2A) BUILD MODEL — LOCKED SETUP SPEC + CHECKLIST

### Locked candidate models
- [x] Model A: DNM + CNM (blockwise threshold).
- [x] Model B: DNM + CNM (blockwise asymptote).
- [x] Model C: DNM + DDM (`x0_t = k_z * psi_t`, `v_t = k_v * LLR_t`).

### Locked numerical policy
- [x] `DT = 1 ms`
- [x] `T_MAX = 5000 ms`
- [x] `N_SIMS_PER_TRIAL = 2000`
- [x] `RT_BIN_WIDTH = 20 ms`
- [x] `EPS = 1e-12`
- [x] Deterministic seed policy is fixed.


In [ ]:
# STEP 2A — SETUP, IMPORTS, PATHS, AND LOCKED CONSTANTS

from pathlib import Path
import importlib
import sys
from typing import Final

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

# Resolve repository root so notebook paths work from different launch folders.
REPO_ROOT = Path.cwd().resolve().parent.parent
if not (REPO_ROOT / "src").exists() and (REPO_ROOT.parent / "src").exists():
    REPO_ROOT = REPO_ROOT.parent

# Add `src/` and `src/elias/` to the import path for local modules.
SRC_ROOT = REPO_ROOT / "src"
ELIAS_SRC = SRC_ROOT / "elias"
for import_path in (SRC_ROOT, ELIAS_SRC):
    if str(import_path) not in sys.path:
        sys.path.insert(0, str(import_path))

# Reload local module so notebook edits are picked up without stale imports.
import elias_models as _elias_models
importlib.reload(_elias_models)

# Import Elias-local preprocessing helpers and model runners.
from elias_models import (
    load_participant_data,
    preprocess_loaded_participant_data,
    run_all_models_for_participant,
    run_model_a_threshold,
    run_model_b_asymptote,
    run_model_c_ddm,
)
# ------------------------------------------------------------------------------
# 4) Locked hard-coded constants (must match MODEL-COMPARISON LOCK).
# Keep these fixed across models/participants for fair comparison.
# `elias_models` APIs now use milliseconds directly.


# Simulation integration step (DT = delta time) in milliseconds.
DT: Final[float] = 1

# Maximum allowed RT window per trial in milliseconds.
T_MAX: Final[float] = 5000

# Monte Carlo samples per trial for probability estimates.
N_SIMS_PER_TRIAL: Final[int] = 2000

# RT histogram bin width in ms for density scoring.
RT_BIN_WIDTH: Final[float] = 20

# Small smoothing constant (EPS=epsilon) to avoid log(0) in likelihood terms.
EPS: Final[float] = 1e-12

# Global base RNG seed for reproducible simulation/scoring.
SEED: Final[int] = 0
# ------------------------------------------------------------------------------

print("Setup complete.")
print(f"REPO_ROOT = {REPO_ROOT}")


## STEP 2B) BUILD MODEL — DATA/PREPROCESSING LOCK + CHECKLIST

### Locked preprocessing/data rules
- [x] Source data uses `data/elias.csv`, `data/evan.csv`, `data/maik.csv` merged into `participants.csv`.
- [x] Exclusion rules are fixed: drop missing required values, RT < 150 ms, RT > 5000 ms.
- [x] Units remain milliseconds end-to-end.
- [x] Split rule is fixed: TRAIN `trial_index <= 30`, TEST `trial_index >= 31`.


In [ ]:
# STEP 2B — DATA LOADING, EXCLUSIONS, PREPROCESSING, AND SPLIT LABELS

# Use the existing merged participant CSV stored in `data/`.
participants_csv_path = REPO_ROOT / "data" / "participants.csv"

# Load participant rows and derive model-ready state columns.
df_loaded = load_participant_data(
    csv_path=participants_csv_path,
    participant_ids=None,
    hazard_col="subjective_h_snapshot",
    reset_on=("participant", "block"),
)

# Apply shared preprocessing helper to keep this notebook cell concise.
prep_outputs = preprocess_loaded_participant_data(
    df_loaded,
    min_rt_ms=150,
    max_rt_ms=5000,
    train_trial_max_index=30,
    expected_blocks_per_participant=4,
    nominal_trials_per_block_before=40,
)

# Unpack outputs used in later notebook sections.
df_all = prep_outputs["df_all"]
removed_rows_df = prep_outputs["removed_rows_df"]
preprocessing_overview_table = prep_outputs["preprocessing_overview_table"]
participant_structure_table = prep_outputs["participant_structure_table"]

# Report whether this safety step changed anything.
print(f"Participants CSV path:    {participants_csv_path}")
print(f"Rows before safety check: {prep_outputs['before_n']}")
print(f"Rows after safety check:  {prep_outputs['after_n']}")
print(f"Rows removed:             {prep_outputs['removed_n']}")
print(f"Safety check changed data: {prep_outputs['safety_check_changed_data']}")

# Show participants still present after exclusions.
print("\nParticipants:\n", df_all["participant_id"].unique())

# Show participant-level structure checks requested in section 2.
print("\nParticipant structure checks:")
display(participant_structure_table)

# Show preprocessing overview table requested in section 2.
print("\nPreprocessing overview table:")
display(preprocessing_overview_table)

# Show all rows removed by validity/RT filters for auditability.
print("\nRemoved rows (all rows excluded by validity/RT checks):")
display(removed_rows_df)

# Show the prepared modeling DataFrame used by downstream cells.
print("\nPrepared modeling DataFrame (head):")
display(df_all.head())


### STEP 2B RESULTS — CURRENT DATA SNAPSHOT

- Using the current merged file `data/participants.csv`, the dataset contains `3` participants with `467` rows before exclusions:
  - `P01`: `160` rows
  - `P02`: `147` rows (short block 1)
  - `P03`: `160` rows
- Block structure before exclusions:
  - `P01`: `40/40/40/40`
  - `P02`: `27/40/40/40`
  - `P03`: `40/40/40/40`
- After locked exclusions (`drop missing required values`, `RT < 150 ms`, `RT > 5000 ms`), `456` rows remain and `11` rows are removed.
- Removal reasons in current data:
  - `10` rows with `RT > 5000 ms`
  - `1` row with `RT < 150 ms`
  - `0` rows dropped due to missing/non-finite required values
- Participant-level counts after exclusions:
  - `P01`: `149/160` kept (`11` dropped)
  - `P02`: `147/147` kept (`0` dropped)
  - `P03`: `160/160` kept (`0` dropped)
- Split consequences with trial-index rule:
  - `P02` block 1 yields `17` TRAIN and `10` TEST trials (valid, but fewer TRAIN trials than full blocks).


## STEP 2C) BUILD MODEL — UNIFIED SIMULATION-TO-LIKELIHOOD LOCK + CHECKLIST

- [x] Common scoring interface is implemented via `score_model_simulation_likelihood(...)`.
- [x] Per-trial `p(choice_t)` is estimated from simulations.
- [x] Per-trial `p(rt_t | choice_t)` uses histogram density (`20 ms` bins + `EPS` smoothing).
- [x] Joint trial score is implemented: `L_t = -log p(choice_t) - log p(rt_t | choice_t)`.
- [x] Aggregates returned: joint (primary), choice-only, RT-only conditional.


In [ ]:
# STEP 2C — UNIFIED SIMULATION-TO-LIKELIHOOD SCORING DEMO

from elias_models import score_model_simulation_likelihood

# Verify Elias-local choice encoding after import-time normalization.
choice_values = sorted(df_all["choice"].dropna().astype(int).unique().tolist())
print("Observed choice encoding in df_all:", choice_values)

# Use a compact TEST subset for a fast build-step smoke run.
scoring_input_df = df_all[df_all["split"] == "TEST"].copy()
if scoring_input_df.empty:
    scoring_input_df = df_all.copy()
scoring_input_df = scoring_input_df.head(12).copy()

scoring_configs = {
    "cont_threshold": {
        "max_duration_ms": float(T_MAX),
        "dt_ms": float(DT),
    },
    "cont_asymptote": {
        "max_duration_ms": float(T_MAX),
        "dt_ms": float(DT),
    },
    "ddm_dnm": {
        "max_duration_ms": float(T_MAX),
        "dt_ms": float(DT),
    },
}

aggregate_rows: list[dict[str, object]] = []
trial_scores_by_model: dict[str, pd.DataFrame] = {}

for model_name, model_params in scoring_configs.items():
    score_output = score_model_simulation_likelihood(
        scoring_input_df,
        model_name=model_name,
        model_params=model_params,
        n_sims_per_trial=min(N_SIMS_PER_TRIAL, 200),
        rt_bin_width_ms=float(RT_BIN_WIDTH),
        rt_max_ms=float(T_MAX),
        eps=float(EPS),
        random_seed=int(SEED),
    )
    aggregate_rows.append(score_output["aggregate_scores"])
    trial_scores_by_model[model_name] = score_output["trial_scores"]

aggregate_scores_table = (
    pd.DataFrame(aggregate_rows)
    .sort_values("joint_score", ascending=True)
    .reset_index(drop=True)
)

print("\nAggregate simulation-likelihood scores (lower is better):")
display(aggregate_scores_table)

for model_name, trial_scores in trial_scores_by_model.items():
    print(f"\n{model_name} trial-score preview:")
    display(trial_scores.head())


## STEP 2D) BUILD MODEL — PARAMETERIZATION/BOUNDS LOCK + CHECKLIST

- [x] Define explicit parameter vectors and transforms.
- [x] Model A: `theta_A = [thr_b1, thr_b2, thr_b3, thr_b4, t0, g]` with valid bounds.
- [x] Model B: `theta_B = [asy_b1, asy_b2, asy_b3, asy_b4, t0, g]` with valid bounds.
- [x] Model C: `theta_C = [a, t0, k_v, k_z]` with fixed `s = 1.0`.


### IMPLEMENTATION DECISIONS + DOWNSIDES FOR DISCUSSION/RESULTS

- Decision: bounded parameter vectors use logistic box transforms (`eta -> theta`) with finite bounds.
- Decision: block parameters use fixed independent order `b1, b2, b3, b4` (no monotonic coupling).
- Decision: Step 2D provides `theta -> scorer model_params` mapping now; full blockwise runtime integration remains a later step.
- Downside: logistic transforms can saturate near bounds and flatten local gradient information.
- Downside: independent block parameters increase flexibility and can increase overfitting risk.
- Downside: A/B block sidecars are not consumed by current scorer internals, so block-effect evidence is deferred until integration.
- Downside: fixing `s = 1.0` in Model C shifts scale tradeoffs into `a` and `k_v`, reducing direct interpretability.


In [ ]:
# STEP 2D — PARAMETERIZATION, BOUNDS, AND SCORER-MAPPING HELPER DEMO

from elias_models import (
    eta_to_theta,
    get_parameter_spec,
    theta_to_eta,
    theta_to_named_params,
    theta_to_scoring_model_params,
)

model_names = ["cont_threshold", "cont_asymptote", "ddm_dnm"]

# Display ordered parameter specifications used by the optimizer/scoring bridge.
for model_name in model_names:
    print(f"\n{model_name} parameter specification:")
    display(pd.DataFrame(get_parameter_spec(model_name)))

# Use moderate eta values to validate bounded transform and inverse consistency.
eta_examples = {
    "cont_threshold": np.array([-0.8, -0.2, 0.3, 0.9, -0.4, 0.5], dtype=float),
    "cont_asymptote": np.array([0.6, -0.4, 0.1, -0.7, 0.2, -0.3], dtype=float),
    "ddm_dnm": np.array([0.2, -0.5, 0.8, -0.6], dtype=float),
}

roundtrip_rows: list[dict[str, object]] = []

for model_name, eta_vector in eta_examples.items():
    theta_vector = eta_to_theta(model_name, eta_vector)
    eta_roundtrip = theta_to_eta(model_name, theta_vector)
    max_abs_error = float(np.max(np.abs(eta_roundtrip - eta_vector)))

    spec_df = pd.DataFrame(get_parameter_spec(model_name))
    lower = spec_df["lower"].to_numpy(dtype=float)
    upper = spec_df["upper"].to_numpy(dtype=float)
    in_bounds = bool(np.all(theta_vector >= lower) and np.all(theta_vector <= upper))

    named_params = theta_to_named_params(model_name, theta_vector)
    scoring_params = theta_to_scoring_model_params(
        model_name,
        theta_vector,
        block_ids=(1, 2, 3, 4),
    )

    roundtrip_rows.append(
        {
            "model_name": model_name,
            "theta_dim": int(theta_vector.size),
            "max_abs_eta_roundtrip_error": max_abs_error,
            "theta_within_bounds": in_bounds,
            "scoring_keys": ", ".join(sorted(scoring_params.keys())),
        }
    )

    print(f"\n{model_name} named parameters:")
    display(pd.DataFrame([named_params]))
    print(f"{model_name} scorer parameter mapping:")
    display(pd.DataFrame([scoring_params]))

roundtrip_table = pd.DataFrame(roundtrip_rows)
print("\nTransform roundtrip and bounds validation summary:")
display(roundtrip_table)

print("\nNote: A/B block sidecars are provided for later optimization/runtime integration and are not consumed by current scorer internals.")


## STEP 3) FIT MODEL TO SURROGATE DATA — LOCK + CHECKLIST

### Locked checkpoint rule
- [ ] Simulate surrogate datasets from each candidate generating model using pseudo-true parameters.

### Terminology note
- Surrogate recovery starts with broad pseudo-true parameter coverage across Step 2D bounds.
- Optional post-Step-4 rerun can narrow ranges using participant-informed fitted distributions.

### Step 3 Execution And Retrieval
- Notebook run: set `RUN_STEP3_PIPELINE = True` in the first Step 3 code cell and execute it once.
- CLI run: `PYTHONPATH=src:src/elias python -m elias_models.cli surrogate-run --run-id <run_id> --csv-path data/participants.csv --n-surrogates-per-model <N> --seed <S> --output-root data/elias`
- Show one run: `PYTHONPATH=src:src/elias python -m elias_models.cli surrogate-show --run-id <run_id> --output-root data/elias`
- List runs: `PYTHONPATH=src:src/elias python -m elias_models.cli surrogate-list --output-root data/elias`
- Persistent outputs: `data/elias/surrogate_recovery/runs/<run_id>/tables/`
- Cross-session loading in notebook: `load_step3_run(run_id, output_root="data/elias")`.


In [ ]:
# STEP 3 — BUILD ONE CANONICAL PIPELINE CONFIG AND OPTIONALLY RUN IT

from elias_models import build_step3_pipeline_config, run_step3_pipeline

STEP3_RUN_ID = "step3_surrogate_recovery_v1"
STEP3_OUTPUT_ROOT = REPO_ROOT / "data" / "elias"

# One canonical Step 3 configuration shared by notebook and CLI.
STEP3_CONFIG = build_step3_pipeline_config(
    candidate_models=("cont_threshold", "cont_asymptote", "ddm_dnm"),
    n_surrogates_per_model=20,
    surrogate_n_draws_per_trial=128,
    fit_n_starts=4,
    fit_n_iterations=8,
    fit_n_sims_per_trial=150,
    dt_ms=float(DT),
    max_duration_ms=float(T_MAX),
    random_seed=int(SEED),
    soft_gate_joint_diag_min=0.60,
    soft_gate_param_median_r_min=0.30,
)

cli_models = ",".join(STEP3_CONFIG["candidate_models"])
cli_cmd = (
    f"PYTHONPATH=src:src/elias python -m elias_models.cli surrogate-run "
    f"--run-id {STEP3_RUN_ID} "
    f"--csv-path data/participants.csv "
    f"--candidate-models {cli_models} "
    f"--n-surrogates-per-model {STEP3_CONFIG['n_surrogates_per_model']} "
    f"--surrogate-n-draws-per-trial {STEP3_CONFIG['surrogate_n_draws_per_trial']} "
    f"--fit-n-starts {STEP3_CONFIG['fit_n_starts']} "
    f"--fit-n-iterations {STEP3_CONFIG['fit_n_iterations']} "
    f"--fit-n-sims-per-trial {STEP3_CONFIG['fit_n_sims_per_trial']} "
    f"--dt-ms {STEP3_CONFIG['dt_ms']} "
    f"--max-duration-ms {STEP3_CONFIG['max_duration_ms']} "
    f"--seed {STEP3_CONFIG['random_seed']} "
    f"--output-root {STEP3_OUTPUT_ROOT}"
)

print("Step 3 run id:", STEP3_RUN_ID)
print("Step 3 output root:", STEP3_OUTPUT_ROOT)
print("\nStep 3 canonical config:")
print(STEP3_CONFIG)
print("\nEquivalent CLI command:")
print(cli_cmd)

# Keep False during code editing; switch to True on the faster machine to execute the full run.
RUN_STEP3_PIPELINE = False

if RUN_STEP3_PIPELINE:
    step3_run_output = run_step3_pipeline(
        df_all,
        run_id=STEP3_RUN_ID,
        output_root=STEP3_OUTPUT_ROOT,
        config=STEP3_CONFIG,
        overwrite=False,
    )
    print("\nStep 3 pipeline run completed.")
    print("Run directory:", step3_run_output["run_dir"])
    print("Soft-gate status:", step3_run_output["manifest"]["soft_gate"]["overall_status"])
else:
    step3_run_output = None
    print("\nStep 3 pipeline execution skipped. Set RUN_STEP3_PIPELINE=True to execute.")


- [ ] Refit all candidate models on each surrogate dataset.

In [ ]:
# STEP 3 — LOAD PERSISTED REFIT RESULTS AND DISPLAY FIT DIAGNOSTICS

from elias_models import load_step3_run

required_names = ["STEP3_RUN_ID", "STEP3_OUTPUT_ROOT", "STEP3_CONFIG"]
missing_names = [name for name in required_names if name not in globals()]
if missing_names:
    raise ValueError(
        f"Missing required Step 3 configuration variables: {missing_names}. "
        "Run the first Step 3 code cell first."
    )

step3_loaded_run = load_step3_run(
    run_id=STEP3_RUN_ID,
    output_root=STEP3_OUTPUT_ROOT,
)

step3_fit_results = step3_loaded_run["tables"].get("fit_results", pd.DataFrame())
if step3_fit_results.empty:
    raise ValueError(
        "fit_results table is empty. Execute Step 3 pipeline first "
        "(notebook RUN_STEP3_PIPELINE=True or CLI surrogate-run)."
    )

step3_fit_summary_table = (
    step3_fit_results.groupby(["generating_model_name", "candidate_model_name"], as_index=False)
    .agg(
        n_rows=("joint_score", "size"),
        joint_min=("joint_score", "min"),
        joint_median=("joint_score", "median"),
    )
    .sort_values(["generating_model_name", "joint_median", "candidate_model_name"])
    .reset_index(drop=True)
)

step3_joint_scores_finite = bool(np.isfinite(step3_fit_results["joint_score"]).all())
expected_rows = int(
    len(STEP3_CONFIG["candidate_models"])
    * int(STEP3_CONFIG["n_surrogates_per_model"])
    * len(STEP3_CONFIG["candidate_models"])
)
actual_rows = int(len(step3_fit_results))

print("Loaded run directory:", step3_loaded_run["run_dir"])
print(
    f"Refit rows: {actual_rows} (expected {expected_rows}) | "
    f"All joint scores finite: {step3_joint_scores_finite}"
)

print("\nStep 3 chunk-2 diagnostic summary:")
display(step3_fit_summary_table)

print("\nStep 3 chunk-2 fit results preview:")
display(step3_fit_results.head())


- [ ] Build model-recovery matrix and parameter-recovery summary.

In [ ]:
# STEP 3 — LOAD RECOVERY MATRICES AND PARAMETER-RECOVERY SUMMARIES

from elias_models import load_step3_run

if "step3_loaded_run" not in globals() or step3_loaded_run is None:
    step3_loaded_run = load_step3_run(
        run_id=STEP3_RUN_ID,
        output_root=STEP3_OUTPUT_ROOT,
    )

step3_model_recovery_joint_counts = step3_loaded_run["tables"].get(
    "model_recovery_joint_counts",
    pd.DataFrame(),
)
step3_model_recovery_joint_rates = step3_loaded_run["tables"].get(
    "model_recovery_joint_rates",
    pd.DataFrame(),
)
step3_model_recovery_bic_counts = step3_loaded_run["tables"].get(
    "model_recovery_bic_counts",
    pd.DataFrame(),
)
step3_model_recovery_bic_rates = step3_loaded_run["tables"].get(
    "model_recovery_bic_rates",
    pd.DataFrame(),
)
step3_parameter_recovery_summary = step3_loaded_run["tables"].get(
    "parameter_recovery_summary",
    pd.DataFrame(),
)

print("Step 3 chunk-3 model recovery (joint counts):")
display(step3_model_recovery_joint_counts)

print("\nStep 3 chunk-3 model recovery (joint rates):")
display(step3_model_recovery_joint_rates)

print("\nStep 3 chunk-3 model recovery (BIC counts):")
display(step3_model_recovery_bic_counts)

print("\nStep 3 chunk-3 model recovery (BIC rates):")
display(step3_model_recovery_bic_rates)

print("\nStep 3 chunk-3 parameter-recovery summary:")
display(step3_parameter_recovery_summary)


- [ ] Apply soft-gate interpretation: if recovery is weak, continue but downgrade winner claims.

In [ ]:
# STEP 3 — LOAD SOFT-GATE SUMMARY AND PRINT INTERPRETATION

from elias_models import load_step3_run

if "step3_loaded_run" not in globals() or step3_loaded_run is None:
    step3_loaded_run = load_step3_run(
        run_id=STEP3_RUN_ID,
        output_root=STEP3_OUTPUT_ROOT,
    )

step3_soft_gate_summary = step3_loaded_run["tables"].get("soft_gate_summary", pd.DataFrame())
step3_soft_gate_status = step3_loaded_run["manifest"].get("soft_gate", {}).get(
    "overall_status",
    "unknown",
)

print("Step 3 chunk-4 soft-gate summary:")
display(step3_soft_gate_summary)

if step3_soft_gate_status == "pass":
    print("\nSoft-gate interpretation: pass. Recovery quality supports stronger model-winner claims.")
elif step3_soft_gate_status == "caution":
    print("\nSoft-gate interpretation: caution. Winner claims should be qualified with recovery limits.")
elif step3_soft_gate_status == "weak":
    print("\nSoft-gate interpretation: weak. Winner claims should be downgraded and treated as exploratory.")
else:
    print("\nSoft-gate interpretation: unknown. Inspect run artifacts and thresholds.")


## STEP 4) FIT MODEL TO PARTICIPANT DATA — LOCK + CHECKLIST

### Locked evaluation protocol
- [ ] Fit on TRAIN only and evaluate on TEST only.
- [ ] Use one optimizer protocol across models (multi-start count, convergence, iterations, seed policy).
- [ ] Compute per participant: TEST joint total, blockwise consistency, secondary choice/RT totals.
- [ ] Apply locked participant-level and group-level winner rules.


In [ ]:
# STEP 4 — PARTICIPANT FITTING AND TEST EVALUATION IMPLEMENTATION (PENDING)


## STEP 5) LATENT VARIABLE ANALYSIS AND REPORTING — LOCK + CHECKLIST

### Locked reporting requirements
- [ ] Run posterior predictive checks per participant/block.
- [ ] Run change-point/hazard signature checks.
- [ ] Report interpretable latent-variable trajectories/quantities.
- [ ] Apply recovery-aware final conclusions and include hazard-input caveat sentence.


In [ ]:
# STEP 5 — LATENT-VARIABLE ANALYSIS AND REPORTING IMPLEMENTATION (PENDING)
